<a href="https://colab.research.google.com/github/ErikaRochadeAraujo/3D_Unet_com_Perda_Personalizada/blob/main/3D_Unet_com_Perda_Personalizada_visualiza%C3%A7%C3%A3o2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmentação de Imagens de Ressonância Magnética de Tumor Cerebral Utilizando o Modelo 3D U-Net**



# Parte do código que apresenta a visualização do cérebro em pixel com zoom e movimento pela manipulação do usuário

#**Ver arquivo Video3D**



In [ ]:
!pip install config

In [ ]:
import time
import config
import albumentations as A
import nibabel as nib
class ImageReader:
    def __init__(self,data_root, img_size:int=256, normalize:bool=False, single_class:bool=False):
        pad_size = 256 if img_size > 256 else 224
        self.resize = A.Compose(
            [
                A.PadIfNeeded(min_height=pad_size, min_width=pad_size, value=0),
                A.Resize(img_size, img_size)
            ]
        )
        self.normalize=normalize
        self.single_class=single_class
        self.root=data_root

    def read_file(self, path:str) -> dict:
        scan_type = path.split('_')[-1]
        raw_image = nib.load(path).get_fdata()
        raw_mask = nib.load(path.replace(scan_type, 'seg.nii')).get_fdata()
        processed_frames, processed_masks = [], []
        for frame_idx in range(raw_image.shape[2]):
            frame = raw_image[:, :, frame_idx]
            mask = raw_mask[:, :, frame_idx]
            if self.normalize:
                if frame.max() > 0:
                    frame = frame/frame.max()
                frame = frame.astype(np.float32)
            else:
                frame = frame.astype(np.uint8)
            resized = self.resize(image=frame, mask=mask)
            processed_frames.append(resized['image'])
            processed_masks.append(1*(resized['mask'] > 0) if self.single_class else resized['mask'])
        return {
            'scan': np.stack(processed_frames, 0),
            'segmentation': np.stack(processed_masks, 0),
            'orig_shape': raw_image.shape
        }

    def load_patient_scan(self, idx:int, scan_type:str='flair') -> dict:
        patient_id = str(1).zfill(3)
        scan_filename = f'{self.root}/BraTS20_Training_00{numcase}_flair.nii'
        return self.read_file(scan_filename)

import plotly.graph_objects as go
import numpy as np
import plotly


def generate_3d_scatter(
    x:np.array, y:np.array, z:np.array, colors:np.array,
    size:int=3, opacity:float=0.2, scale:str='Teal',
    hover:str='skip', name:str='MRI'
) -> go.Scatter3d:
    return go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers', hoverinfo=hover,
        marker = dict(
            size=size, opacity=opacity,
            color=colors, colorscale=scale
        ),
        name=name
    )


class ImageViewer3d():
    def __init__(
        self, reader:ImageReader, mri_downsample:int=10, mri_colorscale:str='Ice'
    ) -> None:
        self.reader = reader
        self.mri_downsample = mri_downsample
        self.mri_colorscale = mri_colorscale

    def load_clean_mri(self, image:np.array, orig_dim:int) -> dict:
        shape_offset = image.shape[1]/orig_dim
        z, x, y = (image > 0).nonzero()
        # only (1/mri_downsample) is sampled for the resulting image
        x, y, z = x[::self.mri_downsample], y[::self.mri_downsample], z[::self.mri_downsample]
        colors = image[z, x, y]
        return dict(x=x/shape_offset, y=y/shape_offset, z=z, colors=colors)
    def load_tumor_segmentation(self, image:np.array, orig_dim:int) -> dict:
        tumors = {}
        shape_offset = image.shape[1]/orig_dim
        # 1/1, 1/3 si 1/5 pixeli pentru clasele tumorii  1(nucleu necrotic), 2(edem) si 4(tumoare de amplificare)
        sampling = {
            1: 1, 2: 3, 4: 5
        }
        for class_idx in sampling:
            z, x, y = (image == class_idx).nonzero()
            x, y, z = x[::sampling[class_idx]], y[::sampling[class_idx]], z[::sampling[class_idx]]
            tumors[class_idx] = dict(
                x=x/shape_offset, y=y/shape_offset, z=z,
                colors=class_idx/4
            )
        return tumors
    def collect_patient_data(self, scan:dict) -> tuple:
        clean_mri = self.load_clean_mri(scan['scan'], scan['orig_shape'][0])
        tumors = self.load_tumor_segmentation(scan['segmentation'], scan['orig_shape'][0])
        markers_created = clean_mri['x'].shape[0] + sum(tumors[class_idx]['x'].shape[0] for class_idx in tumors)

        import pandas as pd                   #For surface points
        from scipy.spatial import Delaunay
        # Combine x, y, and z into a list of (x, y, z) tuples
        points = np.array(list(zip(clean_mri['x'],clean_mri['y'],clean_mri['z'])))
        # Compute the Delaunay triangulation
        triangulation = Delaunay(points)
        # Get the indices of the triangles forming the convex hull
        convex_hull_indices = triangulation.convex_hull
        # Flatten the indices to get a list of all surface points
        surface_indices = np.unique(convex_hull_indices.flatten())
        # Extract the surface points from the original list
        surface_points = points[surface_indices]
        arx = []
        ary = []
        arz = []
        for point in surface_points:
          if(point[2]>50):
            arx.append(point[0])
            ary.append(point[1])
            arz.append(point[2])
    #Shortest path

        mind=9999
        minx=0
        miny=0
        minz=0
        import math

        def calculate_3d_distance(point1, point2):
                x1, y1, z1 = point1
                x2, y2, z2 = point2
                distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
                return distance
        xx= np.mean(tumors[1]['x'])
        yy= np.mean(tumors[1]['y'])
        zz= np.mean(tumors[1]['z'])
        for point in surface_points:
              if(point[2]>45):
                p1=[point[0],point[1],point[2]]
                p2=[xx, yy, zz]
                t = calculate_3d_distance(p1,p2)
                if(mind>t):
                  mind=t
                  minx=point[0]
                  miny=point[1]
                  minz=point[2]

        ax=[minx,xx]
        ay=[miny,yy]
        az=[minz,zz]


        return [
            generate_3d_scatter(**clean_mri, scale=self.mri_colorscale, opacity=0.3, hover='skip', name='Brain MRI'),
            generate_3d_scatter(**tumors[1], opacity=0.90, hover='all', name='Necrotic tumor core'),
            generate_3d_scatter(**tumors[2], opacity=0.05, hover='all', name='Peritumoral invaded tissue'),
            generate_3d_scatter(**tumors[4], opacity=0.30, hover='all', name='GD-enhancing tumor'),
            go.Scatter3d(x=ax, y=ay, z=az, mode='lines+markers', marker=dict(size=12, color='blue'), name='Path for tumor'),
        ], markers_created
    def get_3d_scan(self, patient_idx:int, scan_type:str='flair') -> go.Figure:
        scan = self.reader.load_patient_scan(patient_idx, scan_type)
        data, num_markers = self.collect_patient_data(scan)
        fig = go.Figure(data=data)
        fig.update_layout(
            title=f"[Patient id:{patient_idx}] brain MRI scan ({num_markers} points)",
            legend_title="Pixel class (click to enable/disable)",
            font=dict(
                family="Courier New, monospace",
                size=14,
            ),
            margin=dict(
                l=0,r=0,b=0,t=30
            ),
            legend=dict(itemsizing='constant')
        )
        return fig


data_root = f'{DATA_PATH}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_00{numcase}'
# tumour visualization time starts(tv0)
tv0 = time.time()
reader = ImageReader(data_root, img_size=128, normalize=True, single_class=False)
viewer = ImageViewer3d(reader, mri_downsample=25)

fig = viewer.get_3d_scan(numcase, 'flair')
plotly.offline.iplot(fig)